In [1]:
import pickle
import numpy as np
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu1,floatX=float32"
from PIL import Image
import json
from scipy import ndimage
import math
import random
import cv2
import matplotlib.pyplot as plt
import copy
import csv

from UnariesNet_orien import unariesNet
import MyConfig_orien as MyConfig
import Config as Config
import getCamM
import geoFuncs
import drawLib
import GroupCluster

%load_ext autoreload
%autoreload 2

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 1: GeForce GTX TITAN Z (CNMeM is disabled, cuDNN 5110)
/home/pichen/anaconda2/lib/python2.7/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
unaryModel = unariesNet(load_pretrained = True, training = False)

param save at =  ../Unaries_orien_newGT2/trainedModels/
Preparing room
part root folder:  ../../../../../cvlabdata2/home/pichen/Potentials/Parts/Run_transfer/Salsa_2/
pom_file_path:  ../../../dataset/Salsa/rectangles80x80.pom
img_list from 2000  to 3990
img_path_format:  ../../../../../cvlabdata2/home/pichen/Potentials/Parts/Run_transfer/Salsa_2/c%d/%d.npy
Initializing Unaries Network


VGG/VGGNet.py:46: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  p1 = theano.tensor.signal.pool.pool_2d(c12, ds = (2,2), st = (2,2), ignore_border=True)
VGG/VGGNet.py:46: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  p1 = theano.tensor.signal.pool.pool_2d(c12, ds = (2,2), st = (2,2), ignore_border=True)
VGG/VGGNet.py:50: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  p2 = theano.tensor.signal.pool.pool_2d(c22, ds = (2,2), st = (2,2), ignore_border=True)
VGG/VGGNet.py:50: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  p2 = theano.tensor.signal.pool.pool_2d(c22, ds = (2,2), st = (2,2), ignore_border=True)
VGG/VGGNet.py:55: UserWarning: DEPRECATION: the 

loading pretrained params for bbox detection
../Unaries_orien_newGT2/trainedModels/params_Unaries_55.pickle
../Unaries_orien_newGT2/trainedModels/params_VGG_55.pickle


In [3]:
with open('../../../dataset/Salsa/ffGT.json') as read_file:
    ffGTs = json.load(read_file)

In [4]:
RzTs, rvecs, tvecs, camIs, distCs = getCamM.getCamMatrixs()

rotation [ 2.24455904 -0.08139517  0.57346381] Rz angle =  0.573463809317
rotation [ 2.2346803   0.01945033 -1.54860532] Rz angle =  -1.54860531543
rotation [ 2.21293743  0.0243497  -3.10520375] Rz angle =  -3.10520374762
rotation [ 2.24282364 -0.06320134  2.26464335] Rz angle =  2.26464335169


In [5]:
unaryModel.GT_bodys = np.load('./GT_orien/GT_body_camSpace.npy')
unaryModel.GT_heads = np.load('./GT_orien/GT_head_camSpace.npy')
GTx = np.load('./GT_orien/GT_Xs_camSpace.npy')
GTy = np.load('./GT_orien/GT_Ys_camSpace.npy')
all_detections = np.loadtxt('../../../dataset/Salsa/groundtruth.txt')
# GTx_img = np.load('./GT_orien/GT_X_proj.npy')
# GTy_img = np.load('./GT_orien/GT_Y_proj.npy')

In [6]:
#load GT on ground (data given by SALSA)
bodys = np.loadtxt('./GT_orien/GT_body.txt')
Xs_gt = np.loadtxt('./GT_orien/GT_Xs.txt')
Ys_gt = np.loadtxt('./GT_orien/GT_Ys.txt')

def showGT(gt_line):
    x_frame = Xs_gt[:,gt_line]
    y_frame = Ys_gt[:,gt_line]
    b_frame = bodys[:,gt_line]
    pts_GT_world = np.zeros((3,len(x_frame)))
    pts_GT_world[0,:] = x_frame
    pts_GT_world[1,:] = y_frame
    b_GT_world = geoFuncs.pointsOnUnitWithCenter(pts_GT_world.transpose()[:,0:2], np.array(b_frame)) #input pts should be N*3 or N*2
    groundGT, GTpts_shift, GTend_pts_shift = drawLib.drawLinesOnGround(pts_GT_world.transpose(), [b_GT_world], 15)
#     plt.figure(figsize=(10,10))
#     plt.imshow(groundGT)
#     plt.show()
    return groundGT, GTpts_shift, GTend_pts_shift, np.array(b_frame), b_GT_world#np.concatenate((x_frame.reshape(-1,1), y_frame.reshape(-1,1)), axis=1)

In [8]:
frame_num = 100
start = 0
stride = 2

show=False
showVoting = False

sigma_rs = [1.0, 1.2]
thresholds = [0.5]
k_sizes = [35]

#record result in a csv file
# with open('FFresults.csv','wb') as csvfile:
#     wResult = csv.writer(csvfile)
#     wResult.writerow(['sigma_radius', 'voting_threshold', 'kernel_size', 'using_orien_GT', 'TP', 'FP', 'FN', 'F1', 'precision', 'recall' ])

for sigma_r in sigma_rs:
    for thres in thresholds:
        for k_size in k_sizes:
            TPs = np.zeros((2,1))#for (GT-orientation and Estimated-orientation)
            FPs = np.zeros((2,1))
            FNs = np.zeros((2,1))
            for fid in range(start,start+frame_num*stride, stride):
                gt_line = (Config.img_index_list[fid] - 3) / 45
                if fid%50 < 0: #visualize
                    show=True
                    showVoting=True

                detections = all_detections[:,gt_line]# rectID for all 18 people on the ground floor at given frame
                detections = detections.astype(int)
                ESTOriPts_cam = np.zeros(( len(Config.cameras_list), len(detections), 2 ))#estimated angle vector in different views

                for cam in Config.cameras_list:
                    print 'FID %d, cam %d'%(fid,cam)
                    x = unaryModel.get_rgb(fid, cam)#fid is the index of img_index_list

                    #get GT bbox from pom
                    GTbbox = unaryModel.room.templates_array[(cam*Config.n_parts)+(Config.n_parts-1), detections,:]
                    rois_GT = np.zeros((GTbbox.shape[0],5)).astype(np.single)
                    rois_GT[:,1] = GTbbox[:,1]
                    rois_GT[:,2] = GTbbox[:,0]
                    rois_GT[:,3] = GTbbox[:,3]
                    rois_GT[:,4] = GTbbox[:,2]

                    p_out, estimate_rad = unaryModel.run_func(x,rois_GT)#run function to estimate orientation

                    #==============For social behavior analysis===============
                    #Get people's GT position in cam space
                    pts_x = GTx[cam, gt_line, :]
                    pts_x = pts_x.reshape((-1,1))
                    pts_y = GTy[cam, gt_line, :]
                    pts_y = pts_y.reshape((-1,1))
                    pts = np.concatenate((pts_x, pts_y), axis=1) #N*2
                    #Get people's orientation
                    GTb = unaryModel.GT_bodys[cam, gt_line, :]
                    GTb_vec = geoFuncs.pointsOnUnitWithCenter(pts, GTb)
                    estB_vec = geoFuncs.pointsOnUnitWithCenter(pts, estimate_rad[:,0])

                    #transform to world coordinate
                    RzT = RzTs[cam]
                    pts_cam = np.zeros((3,len(pts_x)))
                    pts_cam[0:2, :] = [pts_x.transpose(), pts_y.transpose()]
                    pts_world = geoFuncs.projBack2World(pts_cam, RzT)

                    ptsB_cam = np.zeros((3,len(estB_vec)))
                    ptsB_cam[0:2, :] = [estB_vec[:,0].transpose(), estB_vec[:,1].transpose()]
                    ptsB_world = geoFuncs.projBack2World(ptsB_cam, RzT)
                    ptsB_w = ptsB_world[0:2, :].transpose()

                    ESTOriPts_cam[cam, :, :] = ptsB_w

                #Average the angles predicted in different views
                aveESTOriPts = np.mean(ESTOriPts_cam, axis=0)
                aveESTOriPts_world = np.concatenate( (aveESTOriPts, np.zeros((aveESTOriPts.shape[0],1))), axis = 1 )
                groundEST, ESTpts_shift, ESTend_pts_shift = drawLib.drawLinesOnGround(pts_world.transpose(), [aveESTOriPts_world], 15)
                # vec to angle
                ori = aveESTOriPts - pts_world.transpose()[:,0:2]
                ang_b = geoFuncs.angle_rads( np.array([1,0]), ori)
                
                #Voting
                voteMaps_e, voteCountMaps_e = GroupCluster.buildMaps(groundEST, ESTpts_shift, ang_b, sigma_r, 0.7, k_size)
                membersEST = GroupCluster.Voting(voteMaps_e, voteCountMaps_e,  show = showVoting)
                
                #Visualize, compare with GT
                groundGT, GTpts_shift, GTend_pts_shift, GT_bodyOri, GTpts_2d = showGT(gt_line)  
                voteMaps, voteCountMaps = GroupCluster.buildMaps(groundGT, GTpts_shift, GT_bodyOri, sigma_r, 0.7, k_size)
                membersGT = GroupCluster.Voting(voteMaps, voteCountMaps,  show = showVoting) 

                if show:
                    #transform pos and vec back to image coordinate
                    bodyPts_world = np.concatenate((GTpts_2d, np.zeros((GTpts_2d.shape[0],1))), axis=1 )
                    for c in Config.cameras_list:
                        RzT = RzTs[c]   
                        rvec = rvecs[c]
                        tvec = tvecs[c]
                        camI = camIs[c]
                        distC = distCs[c]
                        posImg = geoFuncs.world2Img(rvec, tvec, camI, distC, pts_world.transpose())
                        bGtPtsImg = geoFuncs.world2Img(rvec, tvec, camI, distC, bodyPts_world)
                        bEstPtsImg = geoFuncs.world2Img(rvec, tvec, camI, distC, aveESTOriPts_world)
                        #load Img
                        rgbFile = Config.rgb_name_list[c]%(Config.img_index_list[fid])
                        rgbImg = Image.open(rgbFile)
                        rgbImg = np.array(rgbImg)
                        #draw lines
                        GTori_img = drawLib.drawLinesInImg(posImg, bGtPtsImg, rgbImg, (255,0,0))
                        drawLib.pairImgs(groundGT, GTori_img)
                        #plt.imsave('GtOrien_cam%d_fid%d.png'%(fid, c), GTori_img)

                        ESTori_img = drawLib.drawLinesInImg(posImg, bEstPtsImg, rgbImg, (0,255,0))
                        drawLib.pairImgs(groundEST, ESTori_img)
                        #plt.imsave('aveOrien_cam%d_fid%d.png'%(fid, c), ESTori_img)
                    
                    show=False
                    showVoting=False


                #load Fformation GT
                ffGTList = ffGTs[gt_line]

                #Evaluate
                TP, FP, FN = GroupCluster.evaluate(ffGTList,membersGT)
                TPs[0] += TP
                FPs[0] += FP
                FNs[0] += FN
                
                TP, FP, FN = GroupCluster.evaluate(ffGTList,membersEST)
                TPs[1] += TP
                FPs[1] += FP
                FNs[1] += FN
            
            #record evaluation result
            epi = 1.e-17
            precisions = TPs/ (TPs+FNs+epi)
            recalls = TPs/ (TPs+FPs+epi)
            F1 = 2*precisions*recalls/(precisions+recalls+epi)
            print 'sigma_r = %f, threshold = %f, kernel size = %d, F1 score = '%(sigma_r, thres, k_size)
            print F1
            with open('FFresults.csv', 'a') as f:
                wResult = csv.writer(f)
                for UseGT in range(2):
                    for camId in range(TPs.shape[1]):
                        wResult.writerow([sigma_r, thres, k_size, 1-UseGT, TPs[UseGT,camId], FPs[UseGT,camId], FNs[UseGT, camId], F1[UseGT,camId], precisions[UseGT,camId], recalls[UseGT,camId] ])
            

FID 0, cam 0
FID 0, cam 1
FID 0, cam 2
FID 0, cam 3
rest [[1, 15]]
matched [[4, 8, 12, 17], [2, 3, 5, 6, 7, 13], [9, 10, 11, 14, 16, 18]]
rest [[1, 10, 15, 16], [9, 11, 14], [7], [18]]
matched [[4, 8, 12, 17], [2, 3, 5, 6, 13]]
TP, FP, FN 2 4 2
FID 2, cam 0
FID 2, cam 1
FID 2, cam 2
FID 2, cam 3
rest [[1, 15]]
matched [[4, 8, 12, 17], [2, 3, 5, 6, 7, 13], [9, 10, 11, 14, 16, 18]]
rest [[1, 10, 15, 16], [9, 11, 14], [18]]
matched [[4, 8, 12, 17], [2, 3, 5, 6, 7, 13]]
TP, FP, FN 2 3 2
FID 4, cam 0
FID 4, cam 1
FID 4, cam 2
FID 4, cam 3
rest []
matched [[9, 10, 11, 14, 16, 18], [4, 8, 12, 17], [1, 15], [2, 3, 5, 6, 7, 13]]
rest [[15, 16], [18], [7]]
matched [[1, 9, 10, 11, 14], [4, 8, 12, 17], [2, 3, 5, 6, 13]]
TP, FP, FN 3 3 1
FID 6, cam 0
FID 6, cam 1
FID 6, cam 2
FID 6, cam 3
rest []
matched [[9, 10, 11, 14, 16, 18], [4, 8, 12, 17], [1, 15], [2, 3, 5, 6, 7, 13]]
rest [[1, 10, 15, 16], [11, 14, 18], [9]]
matched [[4, 8, 12, 17], [2, 3, 5, 6, 7, 13]]
TP, FP, FN 2 3 2
FID 8, cam 0
FID 8, 

In [16]:
#max region
print 'TPs'
print TPs
print 'FPs'
print FPs
print 'FNs'
print FNs

precisions = TPs/ (TPs+FNs+0.0000005)
recalls = TPs/ (TPs+FPs+0.0000005)

print 'precisions'
print precisions
print 'recalls'
print recalls

F1 = 2*precisions*recalls/(precisions+recalls)

print 'F1'
print F1

TPs
[[ 397.]
 [ 321.]]
FPs
[[  28.]
 [ 200.]]
FNs
[[  2.]
 [ 78.]]
precisions
[[ 0.99498747]
 [ 0.80451128]]
recalls
[[ 0.93411765]
 [ 0.61612284]]
F1
[[ 0.96359223]
 [ 0.69782609]]
